# Aerial and segmented/original BGT viewer

In [ ]:
import errno
from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import os
from skimage.io import imread, imshow

In [ ]:
# Define data directory
data_dir = os.path.join('/Volumes','Elements','output')

try:
    os.listdir(data_dir)
    
    sample_dirs = sorted(next(os.walk(data_dir))[1])
    image = 'aerial_image_full.tif'
    bgt_labels = 'bgt_labels.tif'
    bgt_path = os.path.join('bgt', 'bgt.shp')
    bgt_data = os.path.join('bgt', 'bgt.dbf')
except OSError as e:
    print('Error: Data directory:',data_dir,'not found.')

In [ ]:
# Custom labels and colormap
from matplotlib import colors
import matplotlib as mpl

labelmap = ['undefined',
            'watervlakte',
            'greppel, droge sloot',
            'transitie',
            'waterloop',
            'pand',
            'baan voor vliegverkeer',
            'fietspad',
            'inrit',
            'OV-baan',
            'overweg',
            'parkeervlak',
            'rijbaan autosnelweg',
            'rijbaan autoweg',
            'rijbaan lokale weg',
            'rijbaan regionale weg',
            'ruiterpad',
            'spoorbaan',
            'voetgangersgebied',
            'voetpad',
            'voetpad op trap',
            'woonerf']

cmap =  mpl.colors.ListedColormap([
    'xkcd:pale grey',
    'xkcd:sea blue',    
    'xkcd:powder blue',
    "xkcd:robin's egg",
    'xkcd:azure',
    'xkcd:reddish brown',
    'xkcd:silver',
    'xkcd:macaroni and cheese',
    'xkcd:light blue grey',
    'xkcd:dirty yellow',
    'xkcd:light grey',
    'xkcd:grape',
    'xkcd:ivory', # 1
    'xkcd:puke',
    'xkcd:light tan',
    'xkcd:buff',
    'xkcd:pumpkin',
    'xkcd:charcoal',
    'xkcd:dark sand',
    'xkcd:cement',
    'xkcd:mushroom',
    'xkcd:tan green'
    ])

## Aerial image and segmented BGT viewer

In [ ]:
plt.close("all")
%matplotlib notebook
fig = plt.figure(figsize=(16, 8))
grid = gridspec.GridSpec(1, 2, figure=fig, width_ratios=[1,1.07])

# Aerial Image
ax1 = fig.add_subplot(grid[0, 0])
ax1.get_xaxis().set_ticks([])
ax1.get_yaxis().set_ticks([])

# BGT Labels Image
ax2 = fig.add_subplot(grid[0, 1], sharex=ax1, sharey=ax1)
ax2.get_xaxis().set_ticks([])
ax2.get_yaxis().set_ticks([])

#grid.tight_layout(fig)
sample_selector = widgets.Dropdown(
                    options=sample_dirs,
                    value=sample_dirs[0],
                    description='Sample: ',
                    disabled=False,
                    layout=widgets.Layout(margin = '0px 0px 0px 250px'))

bounds=np.arange(0,22)
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

@interact(sample=sample_selector)
def show_image(sample):
    # Aerial Image
    img = imread(os.path.join(data_dir,sample,image))
    ax1.imshow(img, cmap='gray')
    ax1.set_title(image)
    
    # BGT Labels Image
    labels = imread(os.path.join(data_dir,sample, bgt_labels)).astype(np.uint8)
    ims = ax2.imshow(labels, cmap=cmap)
    ax2.set_title(bgt_labels)
    
    # Colorbar
    divider = make_axes_locatable(ax2)
    cbar_ax = divider.append_axes("right", "5%", pad="2%")
    cbar = fig.colorbar(ims, cax=cbar_ax,  cmap=cmap, norm=norm, 
                        boundaries=[-10] + bounds + [10], ticks=bounds +0.5,
                        spacing='uniform')
    cbar.ax.set_yticklabels(labelmap)
    
    grid.tight_layout(fig)
    plt.show()

## BGT Shape file loader

In [ ]:
import geopandas as gp
shape_path = os.path.join(data_dir, sample_dirs[0],bgt_path)
patch = gp.GeoDataFrame.from_file(shape_path)
patch.head()

In [ ]:
plt.close("all")
import geopandas as gp
%matplotlib inline
plt.rcParams['figure.figsize'] = (18,18)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

from __future__ import print_function
from ipywidgets import interact
import ipywidgets as widgets

def plot_bgt(patch_nr):
    shape_path = os.path.join(data_dir, sample_dirs[patch_nr],bgt_path)
    patch = gp.GeoDataFrame.from_file(shape_path)
    patch.plot(column='category', cmap=cmap)
    plt.axis('off')

interact(plot_bgt, patch_nr=widgets.IntSlider(min=0,max=len(sample_dirs),step=1,value=0), continuous_update=False);